In [1]:
import pandas as pd
import numpy as np

## data prepare

In [2]:
filename = "auto-mpg.data"
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
df = pd.read_csv(filename, delim_whitespace=True, names=column_names)
mpg_cut=(df.mpg.max()+df.mpg.min())/2
print(mpg_cut,df.mpg.max(),df.mpg.min())
df['horsepower']=pd.to_numeric(df['horsepower'],errors='coerce')
df['year']=df['year'].astype(float)
df['mpg'] = pd.cut(df.mpg, bins=[df.mpg.min()-1,mpg_cut,df.mpg.max()], labels=[0,1])
def value_cut(data,feature):
    for feat in feature:
        feat_max=data[feat].max()
        feat_min=data[feat].min()
        feat_cut1=(feat_max+feat_min)/3
        data[feat]=pd.cut(df[feat],bins=[feat_min,feat_cut1,feat_cut1*2,feat_max],labels=[0,1,2])
        print(feat,":",feat_min,feat_cut1,2*feat_cut1,feat_max)
        
    return data
value_cut(df,['displacement','acceleration','weight','horsepower'])
yearmin=df.year.min()
yearmax=df.year.max()
x=(yearmax-yearmin)/4
print(yearmin,yearmax,x)
yearlist=[yearmin-1,yearmin+4,yearmin+8,yearmax]
print(yearlist)
df['year'] = pd.cut(df.year, bins=yearlist, labels=[0,1,2])
df.fillna(method="backfill",inplace=True)
df=df.iloc[:,0:7].astype(int)


27.8 46.6 9.0
displacement : 68.0 174.33333333333334 348.6666666666667 455.0
acceleration : 8.0 10.933333333333332 21.866666666666664 24.8
weight : 1613.0 2251.0 4502.0 5140.0
horsepower : 46.0 92.0 184.0 230.0
70.0 82.0 3.0
[69.0, 74.0, 78.0, 82.0]


In [3]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
0,0,8,1,1,1,1,0
1,0,8,2,1,1,1,0
2,0,8,1,1,1,1,0
3,0,8,1,1,1,1,0
4,0,8,1,1,1,0,0


## build datasets

In [4]:
X = df.iloc[:,1:7].to_numpy()
Y = df.iloc[:,0].to_numpy()

In [5]:
import random
sequence = list(range(len(X)))
random.shuffle(sequence)
X = X[sequence]
Y = Y[sequence]
rate = 0.6
train_len = int(len(X)*rate)
X_train,Y_train,X_test,Y_test = X[:train_len],Y[:train_len],X[train_len:],Y[train_len:]

## model implement

In [6]:
# import numpy as np
from collections import Counter
from easydict import EasyDict as edict

class utils:
    
    @staticmethod
    def splite_dataset_with_feature_value(X,y,feature_dim,feature_value):
        """
        Splite the datasets into two parts
        return X_left,y_left,X_right,y_right
        """
        less_part = X[:,feature_dim]<feature_value
        more_part = X[:,feature_dim]>=feature_value
        return X[less_part],y[less_part],X[more_part],y[more_part]
    
    @staticmethod
    def gini(x):
        """x should be with one dimension.
        return $1-p^2$
        """
        count_x = Counter(x)
        sum_p = 0
        for eachcount in count_x.values():
            sum_p += (eachcount/len(x))**2
        return 1-sum_p
    
    @staticmethod
    def createNode(_dict):
        """
        Using dict to represent node
        """
        return edict(_dict)
    
    @staticmethod
    def findMostCommanValue(x):
        """Find the most frequency elements in an array
        """
        # print(x,Counter(x).most_common(1))
        return Counter(x).most_common(1)[0][0]
    
class my_decision_tree:
    
    def __init__(self,max_depth=3,min_leaf_simples=1):
        self.dtree = None
        self.max_depth = max_depth
        self.min_leaf_simples = min_leaf_simples
    
    def fit(self,X,y):
        self.dtree= self.__create_dtree(X,y,1)
    
    def buildNode(self,feature_dim=None,feature_value=None,gini_rate=None,label=None):
        nodeDict = {"feature_dim":feature_dim,
                   "feature_value":feature_value,
                   "gini":gini_rate,
                   "label":label,
                   "child_left":None,
                   "child_right":None}
        return edict(nodeDict)
    
    def __create_dtree(self,X,y,current_depth=1):
        """
        Recursively build the decision tree
        """
#         if len(y) <= self.min_leaf_simples:
#             return self.buildNode(label=utils.findMostCommanValue(y))
        # s.t. not more than self.max_depth
        if current_depth > self.max_depth:
            return None
        # get current X,y cart value
        feature_dim,feature_value,gini_rate = self.__cart(X,y)
        # s.t. feature_dim exist and gini_rate exist
        if feature_dim == -1 or gini_rate == 0:
            return None
        # construct dtree node
        dnode = self.buildNode(feature_dim,feature_value,gini_rate)
        X_left,y_left,X_right,y_right = utils.splite_dataset_with_feature_value(X,y,feature_dim,feature_value)
        # recursive  binary 
        print(len(y_left))
        dnode.child_left = self.__create_dtree(X_left,y_left,current_depth=current_depth+1) # build left_child
        dnode.child_right = self.__create_dtree(X_right,y_right,current_depth=current_depth+1) # build right_child
        if not dnode.child_left: # leaf 
            label = utils.findMostCommanValue(y_left)
            dnode.child_left=self.buildNode(label=label)
        if not dnode.child_right: # leaf
            label = utils.findMostCommanValue(y_right)
            dnode.child_right =self.buildNode(label=label)
        return dnode

    def predict(self,X):
        res = []
        for eachx in X:
            try:
                res.append(self.__singlePredictCycle(eachx,self.dtree))
            except:
                print(eachx)
        return np.array(res)

    def __singlePredictRec(self,x,dnode):
        """
        Recursively find left and right subtrees
        """
        if dnode.label:
            return dnode.label
        if x[dnode.feature_dim]<dnode.feature_value:
            return self.__singlePredict(x,dnode.child_left)
        else:
            return self.__singlePredict(x,dnode.child_right)
    
    def __singlePredictCycle(self,x,dnode):
        """
        Circularly find the left and right subtrees
        """
        templist = [dnode]
        while templist:
            currentnode = templist.pop()
            if currentnode.label is not None:
                return currentnode.label
            if x[currentnode.feature_dim]<currentnode.feature_value:
                templist.append(currentnode.child_left)
            else:
                templist.append(currentnode.child_right)
        return -1

    def __cart(self,X,y):
        """
        Return best splite parameter
        """
        feature_dim_length = X.shape[1]
        # sample_length = len(X)
        minigini = 1
        feature_dim = -1
        best_feature_value = -1
        for eachfeature in range(feature_dim_length):
            feature_values = np.unique(X[:,eachfeature])
            for index in range(len(feature_values)-1):
                feature_value = feature_values[index] + feature_values[index+1]
                feature_value /= 2
                left = X[:,eachfeature] < feature_value
                right = X[:,eachfeature] >= feature_value
                if len(y[left])<self.min_leaf_simples or len(y[right])<self.min_leaf_simples:# less than min_leaf_simples
                    continue
                tempgini = utils.gini(y[left]) + utils.gini(y[right])
                if tempgini < minigini:
                    minigini = tempgini
                    feature_dim = eachfeature
                    best_feature_value = feature_value
        return feature_dim,best_feature_value,minigini
    

## Train

In [19]:
mytree = my_decision_tree(max_depth=4,min_leaf_simples=4)

In [20]:
mytree.fit(X_train,Y_train)

6
228
224
204


In [21]:
mytree.dtree

{'feature_dim': 4,
 'feature_value': 0.5,
 'gini': 0.454481272294887,
 'label': None,
 'child_left': {'feature_dim': None,
  'feature_value': None,
  'gini': None,
  'label': 0,
  'child_left': None,
  'child_right': None},
 'child_right': {'feature_dim': 2,
  'feature_value': 1.5,
  'gini': 0.4581024930747922,
  'label': None,
  'child_left': {'feature_dim': 3,
   'feature_value': 1.5,
   'gini': 0.4616948341836735,
   'label': None,
   'child_left': {'feature_dim': 1,
    'feature_value': 1.5,
    'gini': 0.47880622837370246,
    'label': None,
    'child_left': {'feature_dim': None,
     'feature_value': None,
     'gini': None,
     'label': 0,
     'child_left': None,
     'child_right': None},
    'child_right': {'feature_dim': None,
     'feature_value': None,
     'gini': None,
     'label': 0,
     'child_left': None,
     'child_right': None}},
   'child_right': {'feature_dim': None,
    'feature_value': None,
    'gini': None,
    'label': 0,
    'child_left': None,
    'chi

## evaluation

In [22]:
acc = sum(mytree.predict(X_test) == Y_test) / len(Y_test)
print("test accuracy:  ",acc)

test accuracy:   0.7625


In [23]:
trainacc = sum(mytree.predict(X_train) == Y_train) / len(Y_train)
print("train accuracy:  ",trainacc)

train accuracy:   0.6596638655462185
